In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.interpolate import interp1d
from tqdm import tqdm
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore')

In [2]:
import zipfile
with zipfile.ZipFile('/content/AIS_2022_03_31.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [3]:
df = pd.read_csv('/content/AIS_2022_03_31.csv')
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
0,367702220,2022-03-31T00:00:01,29.78763,-95.08070,0.1,226.5,340.0,JOE B WARD,NaN,WDI4808,31.0,12.0,21.0,8.0,NaN,57.0,A
1,671226100,2022-03-31T00:00:01,25.77626,-80.20320,3.2,143.7,511.0,RELIANCE II,IMO9221322,5VHS7,79.0,0.0,52.0,12.0,2.5,70.0,A
2,367767250,2022-03-31T00:00:01,29.31623,-94.78829,4.5,228.1,511.0,GLEN K,NaN,WDJ3358,52.0,0.0,0.0,0.0,0.0,52.0,A
3,338327436,2022-03-31T00:00:03,47.29634,-122.42233,0.0,360.0,511.0,COOL KAT,IMO0000000,NaN,36.0,NaN,15.0,3.0,NaN,NaN,B
4,367452810,2022-03-31T00:00:06,29.32824,-94.77391,2.6,319.2,511.0,JOHN W JOHNSON,IMO9602344,WDF4516,60.0,0.0,80.0,19.0,3.0,60.0,A


In [4]:
df.shape

(7167046, 17)

In [5]:
df.isna().sum()

,0
MMSI,0
BaseDateTime,0
LAT,0
LON,0
SOG,0
COG,0
Heading,0
VesselName,17201
IMO,3368784
CallSign,833343


In [6]:
df = df.dropna()

In [7]:
df = df[['BaseDateTime','MMSI','IMO','LAT','LON','SOG','COG']]

In [8]:
df.isna().sum()

,0
BaseDateTime,0
MMSI,0
IMO,0
LAT,0
LON,0
SOG,0
COG,0


In [9]:
df.shape

(1979165, 7)

In [10]:
df.to_csv('./modified_ais_data.csv',index=False)

In [26]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from geopy.point import Point
from tqdm import tqdm
from math import atan2, radians, degrees, sin, cos



def lat_lon_range(df_file):
    """
    Calculate the bounding box for latitude and longitude.

    Parameters:
    - df_file: DataFrame containing 'LAT' and 'LON' columns.

    Returns:
    - min_lat, max_lat, min_lon, max_lon: Integer values defining the bounding box.
    """
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon


def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    """
    Calculate the initial bearing (forward azimuth) between two geodesic points.

    Parameters:
    - start_lat, start_lon: Coordinates of the starting point.
    - end_lat, end_lon: Coordinates of the ending point.

    Returns:
    - bearing: Initial bearing in degrees.
    """
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees


def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points):
    """
    Perform geodesic interpolation between two points.

    Parameters:
    - start_lat, start_lon: Coordinates of the start point.
    - end_lat, end_lon: Coordinates of the end point.
    - num_points: Number of intermediate points to generate.

    Returns:
    - lats, lons: Lists of interpolated latitude and longitude values.
    """
    start_point = Point(start_lat, start_lon)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination(start_point, bearing)
        lats.append(interpolated_point.latitude)
        lons.append(interpolated_point.longitude)

    return lats, lons


def sliding_window_segmentation(data, window_size, step_size, mmsi):
    """
    Perform sliding window segmentation on the given DataFrame.

    Parameters:
    - data: Pandas DataFrame containing the trajectory.
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - mmsi: MMSI of the vessel for this segment.

    Returns:
    - segments: A list of DataFrames, each representing a segmented window.
    """
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments


def lat_lon_range(df_file):
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees

def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points):
    start_point = Point(start_lat, start_lon)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination(start_point, bearing)
        lats.append(interpolated_point.latitude)
        lons.append(interpolated_point.longitude)

    return lats, lons

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments

def prepare_ais_data_with_geodesic_interpolation(
    data, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes in seconds
):
    min_lat, max_lat, min_lon, max_lon = bounding_box

    # Step 1: Apply bounding box filter
    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    # Step 2: Aggregate by MMSI and sort by timestamp
    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    # Step 3: Process each MMSI group
    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Step 4: Interpolate data using geodesic interpolation
        resampled_traj = []

        for i in range(len(group) - 1):
            start_row = group.iloc[i]
            end_row = group.iloc[i + 1]

            # Calculate time difference
            time_diff = (end_row['BaseDateTime'] - start_row['BaseDateTime']).total_seconds()

            if time_diff < min_resample_interval:
                print(f"Skipping interpolation for gap < 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']}")
                resampled_traj.append((start_row['BaseDateTime'], start_row['LAT'], start_row['LON'], start_row['SOG']))
                continue

            num_points = int(time_diff / min_resample_interval) + 1
            print(f"Performing interpolation for gap >= 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']} with {num_points} points")

            # Perform geodesic interpolation
            lats, lons = geodesic_interpolation(
                start_row['LAT'], start_row['LON'],
                end_row['LAT'], end_row['LON'],
                num_points
            )

            # Interpolate SOG linearly
            sog_interp = np.linspace(start_row['SOG'], end_row['SOG'], num_points)

            # Generate timestamps
            timestamps = pd.date_range(
                start=start_row['BaseDateTime'],
                end=end_row['BaseDateTime'],
                periods=num_points
            )

            # Append interpolated points to trajectory
            resampled_traj.extend(zip(timestamps, lats, lons, sog_interp))

        # Ensure the last point is included
        resampled_traj.append((end_row['BaseDateTime'], end_row['LAT'], end_row['LON'], end_row['SOG']))

        resampled_df = pd.DataFrame(resampled_traj, columns=['BaseDateTime', 'LAT', 'LON', 'SOG'])

        # Perform sliding window segmentation
        trajectory_segments = sliding_window_segmentation(resampled_df, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    # Combine all segmented trajectories into a single DataFrame
    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/content/modified_ais_data.csv")
    df = df[:10000]

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes
    )

    # Save segmented trajectories to CSV
    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)
    print("Segmentation completed and saved to 'segmented_trajectories.csv'")


# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/content/modified_ais_data.csv")
    df = df[:10000]

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes
    )

    # Save segmented trajectories to CSV
    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)
    print("Segmentation completed and saved to 'segmented_trajectories.csv'")


Processing MMSI:   0%|          | 7/2003 [00:00<00:37, 53.86MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:32 and 2022-03-31 00:05:35
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:35 and 2022-03-31 00:08:34
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:17 and 2022-03-31 00:01:18
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:18 and 2022-03-31 00:02:36
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:36 and 2022-03-31 00:03:47
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:47 and 2022-03-31 00:04:58
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:58 and 2022-03-31 00:06:16
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:16 and 2022-03-31 00:07:27
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:27 and 2022-03-31 00:08:47
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:08:47 and 2022-03-31 00:09:58
Skipping interpolation for gap < 15 minu

Processing MMSI:   1%|          | 24/2003 [00:00<00:29, 67.82MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:33 and 2022-03-31 00:08:34
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:55 and 2022-03-31 00:04:02
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:04 and 2022-03-31 00:01:10
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:10 and 2022-03-31 00:02:22
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:22 and 2022-03-31 00:03:28
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:28 and 2022-03-31 00:04:34
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:34 and 2022-03-31 00:05:40
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:40 and 2022-03-31 00:06:46
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:46 and 2022-03-31 00:07:53
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:53 and 2022-03-31 00:08:58
Skipping interpolation for gap < 15 minu

Processing MMSI:   3%|▎         | 61/2003 [00:00<00:14, 131.59MMSI/s]

...

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:53 and 2022-03-31 00:03:53
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:53 and 2022-03-31 00:06:53
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:17 and 2022-03-31 00:05:18
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:36 and 2022-03-31 00:05:31
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:05 and 2022-03-31 00:04:52
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:52 and 2022-03-31 00:06:16
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:16 and 2022-03-31 00:07:23
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:23 and 2022-03-31 00:09:05
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:36 and 2022-03-31 00:05:36
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:02 and 2022-03-31 00:01:12
Skipping interpolation for gap < 15 minu

Processing MMSI:  99%|█████████▊| 1977/2003 [00:12<00:00, 141.37MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:27 and 2022-03-31 00:06:32
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:32 and 2022-03-31 00:07:39
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:39 and 2022-03-31 00:09:50
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:45 and 2022-03-31 00:05:41
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:41 and 2022-03-31 00:08:47
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:01 and 2022-03-31 00:01:12
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:12 and 2022-03-31 00:02:22
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:22 and 2022-03-31 00:03:32
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:32 and 2022-03-31 00:04:33
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:33 and 2022-03-31 00:05:43
Skipping interpolation for gap < 15 minu

Processing MMSI: 100%|██████████| 2003/2003 [00:12<00:00, 159.37MMSI/s]


Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:30 and 2022-03-31 00:04:32
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:32 and 2022-03-31 00:07:27
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:12 and 2022-03-31 00:04:44
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:44 and 2022-03-31 00:06:35
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:11 and 2022-03-31 00:04:13
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:13 and 2022-03-31 00:06:17
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:02 and 2022-03-31 00:01:07
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:07 and 2022-03-31 00:02:13
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:13 and 2022-03-31 00:03:20
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:20 and 2022-03-31 00:04:26
Skipping interpolation for gap < 15 minu

In [28]:
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,MMSI,SegmentID
0,2022-03-31 00:00:01,26.09387,-80.08081,28.3,209513000,0
1,2022-03-31 00:01:03,26.09386,-80.08645,14.9,209513000,0
2,2022-03-31 00:02:05,26.09392,-80.09055,11.2,209513000,0
3,2022-03-31 00:03:07,26.09386,-80.09383,10.0,209513000,0
4,2022-03-31 00:04:09,26.09373,-80.09716,10.0,209513000,0


### COG

In [29]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from geopy.point import Point
from tqdm import tqdm
from math import atan2, radians, degrees, sin, cos



def lat_lon_range(df_file):
    """
    Calculate the bounding box for latitude and longitude.

    Parameters:
    - df_file: DataFrame containing 'LAT' and 'LON' columns.

    Returns:
    - min_lat, max_lat, min_lon, max_lon: Integer values defining the bounding box.
    """
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon


def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    """
    Calculate the initial bearing (forward azimuth) between two geodesic points.

    Parameters:
    - start_lat, start_lon: Coordinates of the starting point.
    - end_lat, end_lon: Coordinates of the ending point.

    Returns:
    - bearing: Initial bearing in degrees.
    """
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees


def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points):
    """
    Perform geodesic interpolation between two points.

    Parameters:
    - start_lat, start_lon: Coordinates of the start point.
    - end_lat, end_lon: Coordinates of the end point.
    - num_points: Number of intermediate points to generate.

    Returns:
    - lats, lons: Lists of interpolated latitude and longitude values.
    """
    start_point = Point(start_lat, start_lon)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination(start_point, bearing)
        lats.append(interpolated_point.latitude)
        lons.append(interpolated_point.longitude)

    return lats, lons


def sliding_window_segmentation(data, window_size, step_size, mmsi):
    """
    Perform sliding window segmentation on the given DataFrame.

    Parameters:
    - data: Pandas DataFrame containing the trajectory.
    - window_size: The number of data points in each sliding window.
    - step_size: The step size for the sliding window.
    - mmsi: MMSI of the vessel for this segment.

    Returns:
    - segments: A list of DataFrames, each representing a segmented window.
    """
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments


def lat_lon_range(df_file):
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon):
    start_lat, start_lon, end_lat, end_lon = map(radians, [start_lat, start_lon, end_lat, end_lon])
    delta_lon = end_lon - start_lon
    x = sin(delta_lon) * cos(end_lat)
    y = cos(start_lat) * sin(end_lat) - sin(start_lat) * cos(end_lat) * cos(delta_lon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees

def geodesic_interpolation(start_lat, start_lon, end_lat, end_lon, num_points):
    start_point = Point(start_lat, start_lon)
    total_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
    bearing = calculate_initial_bearing(start_lat, start_lon, end_lat, end_lon)

    lats, lons = [], []
    for frac in np.linspace(0, 1, num_points):
        distance_to_point = total_distance * frac
        interpolated_point = geodesic(meters=distance_to_point).destination(start_point, bearing)
        lats.append(interpolated_point.latitude)
        lons.append(interpolated_point.longitude)

    return lats, lons

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi  # Explicitly set MMSI for each segment
        segments.append(window)
    return segments

def prepare_ais_data_with_geodesic_interpolation(
    data, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes in seconds
):
    min_lat, max_lat, min_lon, max_lon = bounding_box

    # Step 1: Apply bounding box filter
    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    # Step 2: Aggregate by MMSI and sort by timestamp
    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    # Step 3: Process each MMSI group
    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Step 4: Interpolate data using geodesic interpolation
        resampled_traj = []

        for i in range(len(group) - 1):
            start_row = group.iloc[i]
            end_row = group.iloc[i + 1]

            # Calculate time difference
            time_diff = (end_row['BaseDateTime'] - start_row['BaseDateTime']).total_seconds()

            if time_diff < min_resample_interval:
                print(f"Skipping interpolation for gap < 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']}")
                resampled_traj.append((start_row['BaseDateTime'], start_row['LAT'], start_row['LON'], start_row['SOG'], start_row['COG']))
                continue

            num_points = int(time_diff / min_resample_interval) + 1
            print(f"Performing interpolation for gap >= 15 minutes between {start_row['BaseDateTime']} and {end_row['BaseDateTime']} with {num_points} points")

            # Perform geodesic interpolation
            lats, lons = geodesic_interpolation(
                start_row['LAT'], start_row['LON'],
                end_row['LAT'], end_row['LON'],
                num_points
            )

            # Interpolate SOG linearly
            sog_interp = np.linspace(start_row['SOG'], end_row['SOG'], num_points)
            cog_interp = np.linspace(start_row['COG'], end_row['COG'], num_points)


            # Generate timestamps
            timestamps = pd.date_range(
                start=start_row['BaseDateTime'],
                end=end_row['BaseDateTime'],
                periods=num_points
            )

            # Append interpolated points to trajectory
            resampled_traj.extend(zip(timestamps, lats, lons, sog_interp))

        # Ensure the last point is included
        resampled_traj.append((end_row['BaseDateTime'], end_row['LAT'], end_row['LON'], end_row['SOG'], end_row['COG']))

        resampled_df = pd.DataFrame(resampled_traj, columns=['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG'])

        # Perform sliding window segmentation
        trajectory_segments = sliding_window_segmentation(resampled_df, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    # Combine all segmented trajectories into a single DataFrame
    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/content/modified_ais_data.csv")
    df = df[:10000]

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes
    )

    # Save segmented trajectories to CSV
    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)
    print("Segmentation completed and saved to 'segmented_trajectories.csv'")


# Example usage
if __name__ == "__main__":
    # Load AIS data
    df = pd.read_csv("/content/modified_ais_data.csv")
    df = df[:10000]

    # Calculate bounding box
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    # Define window size and step size
    window_size = 10
    step_size = 5

    # Process AIS data
    segmented_trajectories_df = prepare_ais_data_with_geodesic_interpolation(
        df, bounding_box, window_size, step_size, min_resample_interval=900  # 15 minutes
    )

    # Save segmented trajectories to CSV
    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)
    print("Segmentation completed and saved to 'segmented_trajectories.csv'")


Processing MMSI:   1%|▏         | 26/2003 [00:00<00:14, 131.80MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:32 and 2022-03-31 00:05:35
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:35 and 2022-03-31 00:08:34
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:17 and 2022-03-31 00:01:18
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:18 and 2022-03-31 00:02:36
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:36 and 2022-03-31 00:03:47
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:47 and 2022-03-31 00:04:58
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:58 and 2022-03-31 00:06:16
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:16 and 2022-03-31 00:07:27
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:27 and 2022-03-31 00:08:47
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:08:47 and 2022-03-31 00:09:58
Skipping interpolation for gap < 15 minu

Processing MMSI:   3%|▎         | 55/2003 [00:00<00:14, 136.16MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:29 and 2022-03-31 00:01:33
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:33 and 2022-03-31 00:02:57
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:57 and 2022-03-31 00:04:01
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:01 and 2022-03-31 00:05:13
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:13 and 2022-03-31 00:07:41
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:00 and 2022-03-31 00:01:13
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:13 and 2022-03-31 00:02:18
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:18 and 2022-03-31 00:03:24
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:24 and 2022-03-31 00:04:25
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:25 and 2022-03-31 00:05:31
Skipping interpolation for gap < 15 minu

Processing MMSI:   3%|▎         | 69/2003 [00:00<00:14, 129.69MMSI/s]

...

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:57 and 2022-03-31 00:09:18
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:43 and 2022-03-31 00:01:54
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:54 and 2022-03-31 00:03:34
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:34 and 2022-03-31 00:05:05
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:05 and 2022-03-31 00:07:24
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:07:24 and 2022-03-31 00:09:03
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:02 and 2022-03-31 00:03:58
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:58 and 2022-03-31 00:07:02
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:08 and 2022-03-31 00:01:20
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:20 and 2022-03-31 00:02:28
Skipping interpolation for gap < 15 minu

Processing MMSI:  98%|█████████▊| 1971/2003 [00:15<00:00, 178.44MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:57 and 2022-03-31 00:03:57
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:00 and 2022-03-31 00:04:40
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:40 and 2022-03-31 00:05:42
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:05:42 and 2022-03-31 00:06:51
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:06:51 and 2022-03-31 00:08:02
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:02:10 and 2022-03-31 00:08:09
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:25 and 2022-03-31 00:06:25
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:00 and 2022-03-31 00:00:00
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:00 and 2022-03-31 00:00:00
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:00 and 2022-03-31 00:01:11
Skipping interpolation for gap < 15 minu

Processing MMSI: 100%|██████████| 2003/2003 [00:15<00:00, 129.30MMSI/s]

Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:16 and 2022-03-31 00:04:15
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:15 and 2022-03-31 00:07:16
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:05 and 2022-03-31 00:06:00
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:01:30 and 2022-03-31 00:04:32
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:32 and 2022-03-31 00:07:27
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:12 and 2022-03-31 00:04:44
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:44 and 2022-03-31 00:06:35
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:03:11 and 2022-03-31 00:04:13
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:04:13 and 2022-03-31 00:06:17
Skipping interpolation for gap < 15 minutes between 2022-03-31 00:00:02 and 2022-03-31 00:01:07
Skipping interpolation for gap < 15 minu

In [31]:
segmented_trajectories_df.isna().sum()

,0
BaseDateTime,0
LAT,0
LON,0
SOG,0
COG,0
MMSI,0
SegmentID,0
geometry,0


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Add temporal features
def add_temporal_features(data, time_col):
    data[time_col] = pd.to_datetime(data[time_col])
    data['Hour'] = data[time_col].dt.hour
    data['Minute'] = data[time_col].dt.minute
    data['Second'] = data[time_col].dt.second
    data['TimeDelta'] = data[time_col].diff().dt.total_seconds().fillna(0)
    return data


# Filter segments with insufficient rows
def filter_short_segments(data, window_size, prediction_horizon, group_col='SegmentID'):
    min_required_rows = window_size + prediction_horizon
    filtered_data = data.groupby(group_col).filter(lambda x: len(x) >= min_required_rows)
    print(f"Filtered data contains {len(filtered_data)} rows after removing short segments.")
    return filtered_data


# Prepare sequences with temporal features and SegmentID
def prepare_sequences(data, window_size, prediction_horizon,
                      group_col='SegmentID', time_col='BaseDateTime'):
    sequences = []
    unique_segments = sorted(data[group_col].unique())

    for segment in tqdm(unique_segments, desc="Processing Segments", unit="segment"):
        # print('************',segment)
        segment_data = data[data[group_col] == segment].reset_index(drop=True)
        num_rows = len(segment_data)
        # print(f"Segment {segment} has {num_rows} rows.")

        if num_rows < window_size + prediction_horizon:
            # print(f"Skipping segment {segment}: Not enough rows.")
            continue

        for start in range(num_rows - window_size - prediction_horizon + 1):
            x_window = segment_data.iloc[start:start+window_size]
            y_window = segment_data.iloc[start+window_size:start+window_size+prediction_horizon]
            sequences.append({
                'X': {
                    'LAT': x_window['LAT'].tolist(),
                    'LON': x_window['LON'].tolist(),
                    'SOG': x_window['SOG'].tolist(),
                    'COG': x_window['COG'].tolist(),
                    'Hour': x_window['Hour'].tolist(),
                    'Minute': x_window['Minute'].tolist(),
                    'Second': x_window['Second'].tolist(),
                    'TimeDelta': x_window['TimeDelta'].tolist(),
                    'SegmentID': x_window['SegmentID'].tolist(),
                },
                'Y': {
                    'LAT': y_window['LAT'].tolist(),
                    'LON': y_window['LON'].tolist(),
                    'SOG': y_window['SOG'].tolist(),
                    'COG': x_window['COG'].tolist(),
                    'Hour': y_window['Hour'].tolist(),
                    'Minute': y_window['Minute'].tolist(),
                    'Second': y_window['Second'].tolist(),
                    'TimeDelta': y_window['TimeDelta'].tolist(),
                    'SegmentID': y_window['SegmentID'].tolist(),
                }
            })
    return sequences


# Split data into train/val/test
def split_train_val_test(data, test_size=0.2, val_size=0.2):
    train_data, temp_data = train_test_split(data, test_size=test_size + val_size, random_state=42, shuffle=True)
    val_data, test_data = train_test_split(temp_data, test_size=test_size / (test_size + val_size), random_state=42, shuffle=True)
    return train_data, val_data, test_data

# def split_train_val_test(data, test_size=0.2, val_size=0.2, group_col='SegmentID'):
#     sorted_segments = sorted(data[group_col].unique())  # Sort SegmentIDs
#     train_segments, temp_segments = train_test_split(sorted_segments, test_size=test_size + val_size, random_state=42)
#     val_segments, test_segments = train_test_split(temp_segments, test_size=test_size / (test_size + val_size), random_state=42)

#     train_data = data[data[group_col].isin(train_segments)]
#     val_data = data[data[group_col].isin(val_segments)]
#     test_data = data[data[group_col].isin(test_segments)]

#     return train_data, val_data, test_data

# def split_train_val_test(data, train_size=0.7, val_size=0.1, test_size=0.2, group_col='SegmentID'):
#     train_data = []
#     val_data = []
#     test_data = []

#     unique_segments = sorted(data[group_col].unique())  # Sort SegmentIDs

#     for segment in unique_segments:
#         segment_data = data[data[group_col] == segment]

#         # Split into train and temp (validation + test)
#         train_segment, temp_segment = train_test_split(
#             segment_data, test_size=(val_size + test_size), random_state=42, shuffle=False
#         )

#         # Further split temp_segment into validation and test
#         val_segment, test_segment = train_test_split(
#             temp_segment, test_size=(test_size / (val_size + test_size)), random_state=42, shuffle=False
#         )

#         # Append splits to their respective lists
#         train_data.append(train_segment)
#         val_data.append(val_segment)
#         test_data.append(test_segment)

#     # Concatenate all segments' splits
#     train_data = pd.concat(train_data).reset_index(drop=True)
#     val_data = pd.concat(val_data).reset_index(drop=True)
#     test_data = pd.concat(test_data).reset_index(drop=True)

#     return train_data, val_data, test_data

# Data preparation pipeline without scaling
def prepare_data_pipeline_no_scaling(data, window_size, prediction_horizon,
                                     test_size=0.2, val_size=0.2):
    # Add temporal features
    print("Adding temporal features...")
    data = add_temporal_features(data, 'BaseDateTime')

    # Filter short segments
    print("Filtering short segments...")
    data = filter_short_segments(data, window_size, prediction_horizon)

    # Split data
    print("Splitting data into train, validation, and test sets...")
    train_data, val_data, test_data = split_train_val_test(data, test_size, val_size)

    # Prepare sequences
    print("Preparing training sequences...")
    train_sequences = prepare_sequences(train_data, window_size, prediction_horizon)

    print("Preparing validation sequences...")
    val_sequences = prepare_sequences(val_data, window_size, prediction_horizon)

    print("Preparing test sequences...")
    test_sequences = prepare_sequences(test_data, window_size, prediction_horizon)

    return train_sequences, val_sequences, test_sequences

# Parameters
window_size = 5
prediction_horizon = 2
test_size = 0.2
val_size = 0.32


# Prepare data
train_sequences, val_sequences, test_sequences = prepare_data_pipeline_no_scaling(
    segmented_trajectories_df, window_size, prediction_horizon, test_size, val_size
)

print(f"Number of training sequences: {len(train_sequences)}")
print(f"Number of validation sequences: {len(val_sequences)}")
print(f"Number of test sequences: {len(test_sequences)}")



Adding temporal features...
Filtering short segments...
Filtered data contains 840 rows after removing short segments.
Splitting data into train, validation, and test sets...
Preparing training sequences...


Processing Segments: 100%|██████████| 84/84 [00:00<00:00, 929.86segment/s]


Preparing validation sequences...


Processing Segments: 100%|██████████| 80/80 [00:00<00:00, 1174.69segment/s]


Preparing test sequences...


Processing Segments: 100%|██████████| 75/75 [00:00<00:00, 1223.50segment/s]

Number of training sequences: 19
Number of validation sequences: 1
Number of test sequences: 0


In [35]:
import numpy as np

# Function to format sequences for Seq2Seq model
def format_sequences(sequences, input_features, output_features, predicted_seq_len):
    """
    Format sequences for Seq2Seq model.

    Parameters:
    - sequences: List of dictionaries with 'X' and 'Y'.
    - input_features: List of feature names for inputs.
    - output_features: List of feature names for outputs.
    - predicted_seq_len: Length of the predicted sequence.

    Returns:
    - encoder_inputs: NumPy array of shape (num_sequences, input_seq_len, num_features).
    - decoder_inputs: NumPy array of shape (num_sequences, output_seq_len, num_features).
    - targets: NumPy array of shape (num_sequences, output_seq_len, num_targets).
    """
    encoder_inputs = []
    decoder_inputs = []
    targets = []

    for idx, seq in enumerate(sequences):
        try:
            # Prepare encoder inputs
            encoder_inputs.append(
                np.array([seq['X'][feature] for feature in input_features]).T
            )

            # Prepare decoder inputs
            y_data = np.array([seq['Y'][feature] for feature in input_features]).T
            decoder_input = np.zeros((predicted_seq_len, len(input_features)))
            decoder_input[:-1, :] = y_data[:-1, :]
            decoder_inputs.append(decoder_input)

            # Prepare targets
            targets.append(
                np.array([seq['Y'][feature] for feature in output_features]).T
            )
        except KeyError as e:
            print(f"Sequence {idx} skipped due to missing feature: {e}")
            continue
        except ValueError as e:
            print(f"Sequence {idx} skipped due to inconsistent sequence length: {e}")
            continue

    try:
        # Convert to NumPy arrays
        encoder_inputs = np.array(encoder_inputs)
        decoder_inputs = np.array(decoder_inputs)
        targets = np.array(targets)
    except ValueError as e:
        print("Error during array conversion. Ensure all sequences have consistent lengths and shapes.")
        raise e

    return encoder_inputs, decoder_inputs, targets

# Function to inspect sequences for debugging
def inspect_sequences(sequences, input_features, output_features):
    """
    Inspect sequences for potential issues in dimensions or missing features.
    """
    for idx, seq in enumerate(sequences):
        try:
            x_data = [seq['X'][feature] for feature in input_features]
            y_data = [seq['Y'][feature] for feature in output_features]
            print(f"Sequence {idx}: X shape = {np.array(x_data).shape}, Y shape = {np.array(y_data).shape}")
        except KeyError as e:
            print(f"Sequence {idx}: Missing feature {e}")
        except ValueError as e:
            print(f"Sequence {idx}: Inconsistent sequence length")

# Example usage of the functions
if __name__ == "__main__":
    # Example sequences (replace with your data)

    input_features = ['LAT', 'LON', 'SOG', 'COG', 'Hour', 'Minute', 'Second', 'TimeDelta']
    output_features = ['LAT', 'LON', 'SOG', 'COG']
    predicted_seq_len = 2

    # Inspect sequences for debugging
    inspect_sequences(train_sequences, input_features, output_features)

    # Format sequences
    train_enc_inputs, train_dec_inputs, train_targets = format_sequences(
        train_sequences, input_features, output_features, predicted_seq_len
    )

    # Print shapes for verification
    print("Training data shapes:")
    print(f"Encoder inputs: {train_enc_inputs.shape}")
    print(f"Decoder inputs: {train_dec_inputs.shape}")
    print(f"Targets: {train_targets.shape}")

    # Check for NaN or Inf in your data
    print("Checking training data for NaN or Inf...")
    print(np.isnan(train_enc_inputs).any(), np.isinf(train_enc_inputs).any())
    print(np.isnan(train_dec_inputs).any(), np.isinf(train_dec_inputs).any())
    print(np.isnan(train_targets).any(), np.isinf(train_targets).any())


Sequence 0: Inconsistent sequence length
Sequence 1: Inconsistent sequence length
Sequence 2: Inconsistent sequence length
Sequence 3: Inconsistent sequence length
Sequence 4: Inconsistent sequence length
Sequence 5: Inconsistent sequence length
Sequence 6: Inconsistent sequence length
Sequence 7: Inconsistent sequence length
Sequence 8: Inconsistent sequence length
Sequence 9: Inconsistent sequence length
Sequence 10: Inconsistent sequence length
Sequence 11: Inconsistent sequence length
Sequence 12: Inconsistent sequence length
Sequence 13: Inconsistent sequence length
Sequence 14: Inconsistent sequence length
Sequence 15: Inconsistent sequence length
Sequence 16: Inconsistent sequence length
Sequence 17: Inconsistent sequence length
Sequence 18: Inconsistent sequence length
Sequence 0 skipped due to inconsistent sequence length: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous par